# Comparación outputs Kaiju - Kraken

## Leer salida de kaiju o kraken

In [109]:
import pandas as pd

def read_files(directory):
    kraken, kaiju = [], []
    for file in os.listdir(directory):
        # print(file)
        if '.tsv' in file:
            f = pd.read_csv(directory + file, delimiter = '\t', header = None)
            if f.shape[1] == 3:
                f.columns = ["c-clasificado","read_ID","tax_ID_kaiju"]
                f.drop(['c-clasificado'], axis=1, inplace = True)
                kaiju.append(f)

            else:
                f.columns = ["c-clasificado", "read_ID","tax_ID_kraken","length_pb","LCA_taxonomic"]
                f.drop(["c-clasificado","length_pb","LCA_taxonomic"], axis=1, inplace = True)
                kraken.append(f)
            
    kraken, kaiju = pd.concat(kraken), pd.concat(kaiju)
        
    return [kraken, kaiju]

In [110]:
PATH = 'data/'
read_files(PATH)

[                                      read_ID  tax_ID_kraken
 0  NB501110:123:H7C3WBGX7:4:23612:19211:20386        1508507
 1  NB501110:123:H7C3WBGX7:4:23612:13340:20390         500639
 2  NB501110:123:H7C3WBGX7:4:23612:15529:20391              0
 3  NB501110:123:H7C3WBGX7:4:23612:17695:20392          55209
 4  NB501110:123:H7C3WBGX7:4:23612:10408:20392          69218
 5  NB501110:123:H7C3WBGX7:4:23612:11635:20393        1378084
 6  NB501110:123:H7C3WBGX7:4:23612:16321:20394        1378084
 7   NB501110:123:H7C3WBGX7:4:23612:9943:20397          91347
 8  NB501110:123:H7C3WBGX7:4:23612:10288:20398        1461261
 9   NB501110:123:H7C3WBGX7:4:23612:9387:20400        1860097
 0    NB501110:123:H7C3WBGX7:1:11101:9813:1065            547
 1    NB501110:123:H7C3WBGX7:1:11101:9032:1067            329
 2    NB501110:123:H7C3WBGX7:1:11101:9958:1067          69218
 3   NB501110:123:H7C3WBGX7:1:11101:22148:1069        1076549
 4   NB501110:123:H7C3WBGX7:1:11101:17061:1072         487316
 5    NB

## Combinar salidas de archivos

In [111]:
import os
PATH = 'data/'

outputs = read_files(PATH)

# outputs = [read_file('data/10lineas_kraken.tsv'), read_file('data/10lineascola_kraken.tsv'), read_file('data/10lineascola_kaiju.tsv'), read_file('data/10lineas_kaiju.tsv')]
def unite(lOutputs):
    DFfusion = lOutputs[0]
    for i in lOutputs:
        DFfusion = pd.merge(DFfusion, i, how='outer')
   
    return DFfusion

DFfusion = unite(outputs) 

In [112]:
DFfusion.head()

,read_ID,tax_ID_kraken,tax_ID_kaiju
0,NB501110:123:H7C3WBGX7:4:23612:19211:20386,1508507.0,NaN
1,NB501110:123:H7C3WBGX7:4:23612:13340:20390,500639.0,NaN
2,NB501110:123:H7C3WBGX7:4:23612:15529:20391,0.0,NaN
3,NB501110:123:H7C3WBGX7:4:23612:17695:20392,55209.0,NaN
4,NB501110:123:H7C3WBGX7:4:23612:10408:20392,69218.0,NaN


## Obtener linajes

In [113]:
from ete3 import NCBITaxa

def get_full_lineages(otus):

    ncbi = NCBITaxa()  

    lineages = {}  

    if 0 in otus:
        lineages.update({0: ""})
        otus.remove(0)
    if 1 in otus:
        lineages.update({1: "root"})
        otus.remove(1)
    if 2 in otus:
        lineages.update({2: "root;Bacteria"})
        otus.remove(2)

    for entrie in otus:
        lineage = ncbi.get_lineage(entrie)                  #returns list of lineage taxids
        names = ncbi.get_taxid_translator(lineage).values() #returns dict in which the taxids of the lineage list become the keys (int) and the translations the values. Error if there is a 0 
        all_names = ";".join(names)
        lineages.update({entrie: all_names})
    
    return lineages
    


In [114]:
def add_lineages(DFfusion):
    kaiju_lineages = get_full_lineages(list(DFfusion['tax_ID_kaiju'].dropna()))
    kraken_lineages = get_full_lineages(list(DFfusion['tax_ID_kraken'].dropna()))
    DFfusion['kaiju_lineage'] = DFfusion['tax_ID_kaiju']
    DFfusion['kraken_lineage'] = DFfusion['tax_ID_kraken']
    DFfusion.replace({'kaiju_lineage':kaiju_lineages, 'kraken_lineage':kraken_lineages}, inplace = True)
    
    return DFfusion

In [116]:
lineages = add_lineages(DFfusion)

In [117]:
lineages.head()

,read_ID,tax_ID_kraken,tax_ID_kaiju,kaiju_lineage,kraken_lineage
0,NB501110:123:H7C3WBGX7:4:23612:19211:20386,1508507.0,NaN,NaN,root;Bacteria;Proteobacteria;Gammaproteobacter...
1,NB501110:123:H7C3WBGX7:4:23612:13340:20390,500639.0,NaN,NaN,root;Bacteria;Enterobacteriaceae;Enterobacter;...
2,NB501110:123:H7C3WBGX7:4:23612:15529:20391,0.0,NaN,NaN,
3,NB501110:123:H7C3WBGX7:4:23612:17695:20392,55209.0,NaN,NaN,root;Bacteria;Proteobacteria;Gammaproteobacter...
4,NB501110:123:H7C3WBGX7:4:23612:10408:20392,69218.0,NaN,NaN,root;Bacteria;Enterobacteriaceae;Enterobacter;...


## Agregar columna de coincidencia de linajes

In [123]:
lineages[(~lineages['tax_ID_kraken'].isna()) & (~lineages['tax_ID_kaiju'].isna())]

,read_ID,tax_ID_kraken,tax_ID_kaiju,kaiju_lineage,kraken_lineage
13,NB501110:123:H7C3WBGX7:1:11101:22148:1069,1076549.0,470934.0,root;Bacteria;Proteobacteria;Gammaproteobacter...,root;Bacteria;Proteobacteria;Gammaproteobacter...


In [141]:
def consensus(lineages_df):
    l = []
    for item in zip(lineages_df['kraken_lineage'].fillna(''), lineages_df['kaiju_lineage'].fillna('')):
        if item[0] != '' and item[1] != '':
            lineage_a = item[0].split(';')
            lineage_b = item[1].split(';')
            lineage_intersection = []
            if len(lineage_a) < len(lineage_b):
                for i in range(len(lineage_a)):
                    if lineage_a[i] == lineage_b[i]:
                        lineage_intersection.append(lineage_a[i])
                    else:
                        break
            else:
                for i in range(len(lineage_b)):
                    if lineage_a[i] == lineage_b[i]:
                        lineage_intersection.append(lineage_b[i])
                    else:
                        break

            l.append(';'.join(lineage_intersection))
        else:
            l.append(np.nan)
        
    lineages_df['consensus'] = l
    return lineages_df
        

In [142]:
consensus(lineages)

,read_ID,tax_ID_kraken,tax_ID_kaiju,kaiju_lineage,kraken_lineage,consensus
0,NB501110:123:H7C3WBGX7:4:23612:19211:20386,1508507.0,NaN,NaN,root;Bacteria;Proteobacteria;Gammaproteobacter...,NaN
1,NB501110:123:H7C3WBGX7:4:23612:13340:20390,500639.0,NaN,NaN,root;Bacteria;Enterobacteriaceae;Enterobacter;...,NaN
2,NB501110:123:H7C3WBGX7:4:23612:15529:20391,0.0,NaN,NaN,,NaN
3,NB501110:123:H7C3WBGX7:4:23612:17695:20392,55209.0,NaN,NaN,root;Bacteria;Proteobacteria;Gammaproteobacter...,NaN
4,NB501110:123:H7C3WBGX7:4:23612:10408:20392,69218.0,NaN,NaN,root;Bacteria;Enterobacteriaceae;Enterobacter;...,NaN
5,NB501110:123:H7C3WBGX7:4:23612:11635:20393,1378084.0,NaN,NaN,root;Bacteria;Proteobacteria;Gammaproteobacter...,NaN
6,NB501110:123:H7C3WBGX7:4:23612:16321:20394,1378084.0,NaN,NaN,root;Bacteria;Proteobacteria;Gammaproteobacter...,NaN
7,NB501110:123:H7C3WBGX7:4:23612:9943:20397,91347.0,NaN,NaN,root;Bacteria;Proteobacteria;Gammaproteobacter...,NaN
8,NB501110:123:H7C3WBGX7:4:23612:10288:20398,1461261.0,NaN,NaN,root;Bacteria;Actinomycetia;Leucobacter;Microc...,NaN
9,NB501110:123:H7C3WBGX7:4:23612:9387:20400,1860097.0,NaN,NaN,root;Bacteria;Moraxellaceae;Acinetobacter;Prot...,NaN
